In [91]:
from pylab import mpl, plt
plt.style.use(plt.style.available[11])
mpl.rcParams['font.family'] = 'serif'
from tvDatafeed import TvDatafeed, Interval
username = ''
password = ''

tvl = TvDatafeed(username, password)
import numpy as np
import pandas as pd



def t_matrix_maker(ticker):
    data=tvl.get_hist(ticker, 'NSE', interval=Interval.in_1_hour, n_bars=300000, fut_contract=None, extended_session=False)
    data['returns'] = (data['close'] - data['close'].shift(1))/data['close'].shift(1)
    returns = data['returns']
    returns = returns.dropna()
    returns = returns.round(2)
    returns = returns.to_list()
    zeros = np.zeros((int(100*(max(returns) - min(returns))) + 1, int(100*(max(returns) - min(returns))) + 1))
    t_matrix = pd.DataFrame(zeros, [i for i in range(int(100*min(returns)), int(100*max(returns)) + 1)], [i for i in range(int(100*min(returns)), int(100*max(returns)) + 1)])
    
    for i in range(len(returns)-1):
        curr = int (100*returns[i])
        next = int (100*returns[i+1])
        t_matrix.loc[curr, next] += 1

    sums = t_matrix.sum()
    t_matrix = t_matrix.T
    to_drop = []
    for i in t_matrix.columns :
        if sums[i] == 0:
            to_drop.append(i)
    t_matrix = t_matrix.drop(columns=to_drop)
    t_matrix = t_matrix.T
    t_matrix = t_matrix.drop(columns=to_drop)

    sums = t_matrix.sum()
    t_matrix = t_matrix.T
    for i in t_matrix.columns:
        t_matrix[i] = t_matrix[i]/sums[i]
    t_matrix = t_matrix.T
    return t_matrix

ERROR:tvDatafeed.main:error while signin


In [98]:
t_matrix = t_matrix_maker('ETERNAL')

In [104]:
exp_returns = {}

In [110]:
for i in t_matrix.index:
    exp_r = 0
    for col in t_matrix.columns:
        exp_r += int(col)*(t_matrix.loc[i, col])
    exp_returns[i] = float(exp_r)/100
    exp_r = 0

In [111]:
exp_returns

{-19: 0.0,
 -11: 0.0,
 -8: -0.005,
 -6: -0.00857142857142857,
 -5: 0.013333333333333332,
 -4: -0.0026086956521739137,
 -3: -0.002321428571428571,
 -2: 0.0018539325842696624,
 -1: 8.955223880597043e-05,
 0: -5.343307507347404e-06,
 1: 0.0007805429864253397,
 2: 0.0,
 3: 0.004657534246575342,
 4: -0.0008333333333333334,
 5: -0.0058333333333333345,
 6: -0.0025,
 7: 0.0,
 8: -0.01,
 9: -0.005,
 10: 0.02,
 11: 0.06,
 14: -0.04,
 15: -0.02}

In [117]:
def reward(state, action, exp_returns):
    returns, weights = state
    return (weights + action)*exp_returns[returns] - 0.003*abs(action)

In [125]:
reward((-5, -1), 2, exp_returns)

0.007333333333333332

In [126]:
def terminal_reward(state, exp_returns):
    _, weights = state
    return reward(state, -weights, exp_returns)

In [132]:
terminal_reward((-8, 1), exp_returns)

-0.003